In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator
from dtaidistance import dtw
import random
from sklearn.preprocessing import normalize
import pickle #to save/load list of selected hens

#plot
import matplotlib.pyplot as plt
import seaborn as sns

#network
import networkx as nx

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import chi2_distance, correct_key
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


2


# Define parameters

In [4]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
#id_run = 'chapter0_final_'
#path_dataoutput = r'G:\VPHI\Welfare\2- Research Projects\OFHE2.OriginsE2\DataOutput'
#path_extracted_data = os.path.join(path_dataoutput,'TrackingSystem') 
#path_extracted_data = os.path.join(path_extracted_data, id_run)
dico_night_hour = config.dico_night_hour
dico_matching = config.dico_matching
li_binmn = config.li_binmn
penalty = config.penalty
dico_window = config.dico_window
path_extracted_data_SNA = config.path_extracted_data_SNA
#create a director if not existing
if not os.path.exists(path_extracted_data_SNA):
    os.makedirs(path_extracted_data_SNA)
print(id_run)

change the configuration file if not done yet!
correctlightschedule_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download variables

In [51]:
#daily variables (one row per (henID, date))
#daily_ALL_Variable_Tranformed
df_daily = pd.read_csv(os.path.join(path_extracted_data, id_run+'_daily_ALL_variables.csv'), sep=';',
                     parse_dates=['level'], dayfirst=True) 
df_daily['DOA'] = df_daily['level'].map(lambda x: (x-dt.datetime(2020,6,3)).days) 
df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: int((x-119)/7)+1)
#df_daily['weeks_in_laying_barn'] = df_daily['DOA'].map(lambda x: math.ceil(x/7))
#first two month seems good from the %of hens not moving plot! and sounds good too (twice longer thatn they need to start moving)
print(df_daily.shape)

#filter by date
df_daily = df_daily[df_daily['level']>dt.datetime(2020,9,29)]

#remove days that are not fully recorded
df_daily['nbr_sec_per_day'] = df_daily['level'].map(lambda x: dico_night_hour[correct_key(x,dico_night_hour)]['nbr_hour']*60*60)
df_daily['is_correct_amount_time'] = df_daily.apply(lambda x: x['nbr_sec_per_day']==x['verification_daily_total_duration'], axis=1)
df_daily[(~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull())][['level', 'HenID', 'Total_number_transition', 'dur_values', 'verification_daily_total_duration','nbr_sec_per_day']]
print(df_daily.shape)
display(df_daily = df_daily[~((~df_daily['is_correct_amount_time'])&(~df_daily['Total_number_transition'].isnull()))])
print(df_daily.shape)

#remove the days where the night had movement recorded but not the days
print(df_daily.shape)
#display(df_daily[df_daily.isna().any(axis=1)])
df_daily = df_daily[~df_daily['verification_daily_total_duration'].isnull()]
print(df_daily.shape)

df_daily['dur_values_normalized'].replace('[nan, nan, nan, nan, nan]','[np.nan,np.nan,np.nan,np.nan,np.nan]', inplace=True)
df_daily['dur_values_normalized'] = df_daily['dur_values_normalized'].map(lambda x: eval(x))
df_daily['duration_last-firsttransition_mn'] = df_daily['duration_last-firsttransition_mn'].astype(float)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\camil\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (103) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(42515, 149)
(42370, 151)
(42370, 151)
(42370, 151)
(32032, 151)
(32032, 151)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean,DOA,weeks_in_laying_barn,nbr_sec_per_day,is_correct_amount_time
0,hen_130,2020-10-04,0.0,140.0,1166.0,79.0,30613.0,31998.0,"[0.0, 140.0, 1166.0, 79.0, 30613.0]","[0.0, 0.004375273454590912, 0.0364397774860928...",...,NaN,NaN,NaN,NaN,NaN,NaN,123,1,32400,False
1,hen_130,2020-10-05,0.0,0.0,0.0,0.0,32400.0,32400.0,"[0.0, 0.0, 0.0, 0.0, 32400.0]","[0.0, 0.0, 0.0, 0.0, 1.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,124,1,32400,True
2,hen_130,2020-10-06,0.0,0.0,780.0,325.0,31295.0,32400.0,"[0.0, 0.0, 780.0, 325.0, 31295.0]","[0.0, 0.0, 0.024074074074074074, 0.01003086419...",...,NaN,NaN,NaN,NaN,NaN,NaN,125,1,32400,True


In [5]:
#download the MLPs vectors during day only! as we compare on same day, we can restrict to the exact daily calendar!
dico_pen_bin_zone_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                 id_run+'dico_pen_bin_zone_level_h_DAILYLEVEL.pkl'), 'rb'))
dico_pen_bin_level_h = pickle.load(open(os.path.join(path_extracted_data, 
                                                     id_run+'dico_pen_bin_level_h_DAILYLEVEL.pkl'), 'rb'))
#dico of hen as keys and pen as values
df_daily['PenID'] = df_daily['PenID'].map(lambda x: 'pen'+(str(int(x))))
dico_h_p = dict(zip(df_daily['HenID'].tolist(), df_daily['PenID'].tolist()))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Compute the distance across pairs of MLPS - same hen, different dates

In [53]:
li_binmn = [30]

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
#compute the CHI2DISTANCE & DTW for each bird with all other birds
#start recording the time it last
START_TIME = time.perf_counter()
li_df = []
#take (subset of) dates and hens that are available (i.e. for which we comptue the variable on)
li_date = set(df_daily['level'].tolist())
li_zone = ['1_Zone', '2_Zone', '3_Zone', '4_Zone', '5_Zone']
for h in tqdm.tqdm(df_daily['HenID'].unique()):
    li_h_date = df_daily[df_daily['HenID']==h]['level'].unique()  #THIS CHNAGE THE date fomat! from dt.datetime to numy.datetime64!
    li_h_date = [pd.to_datetime(x) for x in li_h_date]
    #we use symmetric measures, so we only do each combination once
    for i,d1 in enumerate(li_h_date):

        #lets compute for hen h1 its difference with all other birds
        for d2 in li_h_date[i+1:]:
            
            try:
                ############ Overall aviary usage similarity across days
                l1_chi2 = df_daily[(df_daily['HenID']==h)&(df_daily['level']==d1)]['dur_values_normalized'].values[0]
                l2_chi2 = df_daily[(df_daily['HenID']==h)&(df_daily['level']==d2)]['dur_values_normalized'].values[0]
                dico_ = {'date1':d1, 'date2':d2, 'HenID':h, 'chi2distance':chi2_distance(l1_chi2,l2_chi2, remove_warning=True)}
                #'dur_values_normalized_henId':l1_chi2,'dur_values_normalized_henID2':l2_chi2,

                ############# Specifics' zones usage similarity across days
                for nbr_binmn in li_binmn:
                    #ALL zone together with chi2distance
                    l1_chi2 = dico_pen_bin_level_h[dico_h_p[h]][nbr_binmn][d1][h]
                    l2_chi2 = dico_pen_bin_level_h[dico_h_p[h]][nbr_binmn][d2][h]
                    dico_['nbr_obs_chi2all_'+str(nbr_binmn)] = len(l1_chi2)
                    li_chi2 = [chi2_distance(l1_chi2[i],l2_chi2[i], remove_warning=True) for i in range(0,len(l1_chi2))]
                    dico_['li_chi2_'+str(nbr_binmn)] = li_chi2
                    dico_['chi2distance_ALL_'+str(nbr_binmn)] = np.mean(li_chi2)

                    #per zone with DTW
                    for ZONE in li_zone:
                        dtw_value = np.nan
                        try:
                            l1_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h]][nbr_binmn][ZONE][d1][h], dtype=np.double)
                            l2_dtw = np.array(dico_pen_bin_zone_level_h[dico_h_p[h]][nbr_binmn][ZONE][d2][h], dtype=np.double)
                            #compute distance measure
                            #psi: #of start and end points of a sequence can be ignored if this would lead to a lower distance
                            dtw_value, paths = dtw.warping_paths(l1_dtw, l2_dtw, window=dico_window[nbr_binmn], psi=0, penalty=penalty)
                        except Exception as e:
                            print(e)
                            sys.exit()
                            pass #dont do anything
                        dico_['DTW_'+str(nbr_binmn)+'_'+ZONE] = round(dtw_value,1)  
                        dico_['nbr_obs_l1_'+str(nbr_binmn)+'_'+ZONE] = len(l1_dtw)
                        dico_['nbr_obs_l2_'+str(nbr_binmn)+'_'+ZONE] = len(l2_dtw)
                li_df.append(dico_)
            except Exception as e:
                #pass #dont do anything
                print(e)
                print(h, d1, d2)
                
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))
#turn inter and intra distances into a dataframe                
df_inter = pd.DataFrame(li_df)
#df_inter = df_inter[~df_inter['chi2distance'].isnull()]
df_inter['DTW_30_all'] = df_inter[['DTW_30_1_Zone', 'DTW_30_2_Zone', 'DTW_30_3_Zone','DTW_30_4_Zone','DTW_30_5_Zone']].sum(axis=1)
li_col = list(df_inter.columns)
li_remove = [i for i in li_col if i.startswith('li_chi2_')]
print(li_remove)
li_keep = [i for i in li_col if i not in li_remove]
#df_inter.iloc[0:1000].to_csv(os.path.join(path_extracted_data_visual_adap, id_run+'_df_DistanceBetweenHenSim_4verification.csv'), sep=';', index=False)
#df_inter.filter(li_keep).to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
df_inter.to_csv(os.path.join(path_extracted_data ,id_run+'_df_DistanceINDHenSim_DAILYLEVEL.csv'), sep=';', index=False)
print(df_inter.shape)
display(df_inter.head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  6%|████▍                                                                         | 13/228 [25:35<6:54:47, 115.75s/it]

float division by zero
hen_193 2020-11-24 00:00:00 2020-11-25 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-11-26 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-11-27 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-11-28 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-11-29 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-11-30 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-01 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-02 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-03 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-04 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-05 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-06 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-07 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2020-12-08 00:00:00
float 

float division by zero
hen_193 2020-11-24 00:00:00 2021-04-23 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-04-24 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-04-25 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-04-26 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-04-27 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-04-28 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-04-29 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-04-30 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-05-01 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-05-02 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-05-03 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-05-04 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-05-05 00:00:00
float division by zero
hen_193 2020-11-24 00:00:00 2021-05-06 00:00:00
float 

  7%|█████▉                                                                         | 17/228 [28:04<2:49:46, 48.28s/it]

float division by zero
hen_221 2021-04-13 00:00:00 2021-04-14 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-15 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-16 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-17 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-18 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-19 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-20 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-21 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-22 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-23 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-24 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-25 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-26 00:00:00
float division by zero
hen_221 2021-04-13 00:00:00 2021-04-27 00:00:00
float 

...

 96%|██████████████████████████████████████████████████████████████████████████▌   | 218/228 [4:35:34<11:28, 68.82s/it]

float division by zero
hen_230 2021-04-13 00:00:00 2021-04-14 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-15 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-16 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-17 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-18 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-19 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-20 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-21 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-22 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-23 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-24 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-25 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-26 00:00:00
float division by zero
hen_230 2021-04-13 00:00:00 2021-04-27 00:00:00
float 

100%|██████████████████████████████████████████████████████████████████████████████| 228/228 [4:47:43<00:00, 75.72s/it]


Total running time: 287.72 mn


KeyError: 'henId'

In [62]:
df_inter['DTW_30_all'] = df_inter[['DTW_30_1_Zone', 'DTW_30_2_Zone', 'DTW_30_3_Zone','DTW_30_4_Zone','DTW_30_5_Zone']].sum(axis=1)
li_col = list(df_inter.columns)
li_remove = [i for i in li_col if i.startswith('li_chi2_')]
print(li_remove)
li_keep = [i for i in li_col if i not in li_remove]
#df_inter.iloc[0:1000].to_csv(os.path.join(path_extracted_data_visual_adap, id_run+'_df_DistanceBetweenHenSim_4verification.csv'), sep=';', index=False)
#df_inter.filter(li_keep).to_csv(os.path.join(path_extracted_data_visual_adap ,id_run+'_df_DistanceBetweenHenSim.csv'), sep=';', index=False)
df_inter.to_csv(os.path.join(path_extracted_data ,id_run+'_df_DistanceINDHenSim_DAILYLEVEL.csv'), sep=';', index=False)
print(df_inter.shape)
display(df_inter.head(3))

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['li_chi2_30']
(2796681, 23)


,date1,date2,HenID,chi2distance,nbr_obs_chi2all_30,li_chi2_30,chi2distance_ALL_30,DTW_30_1_Zone,nbr_obs_l1_30_1_Zone,nbr_obs_l2_30_1_Zone,...,DTW_30_3_Zone,nbr_obs_l1_30_3_Zone,nbr_obs_l2_30_3_Zone,DTW_30_4_Zone,nbr_obs_l1_30_4_Zone,nbr_obs_l2_30_4_Zone,DTW_30_5_Zone,nbr_obs_l1_30_5_Zone,nbr_obs_l2_30_5_Zone,DTW_30_all
0,2020-10-04,2020-10-05,hen_130,0.022121,19,"[0.9999999999989999, 0.024765157983628217, 0.0...",0.061993,0.0,19,19,...,14.7,19,19,1.1,19,19,21.9,19,19,40.0
1,2020-10-04,2020-10-06,hen_130,0.005760,19,"[0.2782426778237291, 0.06932523228813971, 0.0,...",0.026351,0.0,19,19,...,5.9,19,19,4.4,19,19,13.1,19,19,25.7
2,2020-10-04,2020-10-07,hen_130,0.009900,19,"[0.661454792656337, 0.024765157983628217, 0.0,...",0.044175,0.0,19,19,...,10.9,19,19,2.4,19,19,13.4,19,19,30.9


# Compute Individual Network

In [ ]:
#TODO!!

In [6]:
path_extracted_data_visual_adap = os.path.join(path_extracted_data,'visual','Treatment&Classs','adaptability')
df_inter = pd.read_csv(os.path.join(path_extracted_data ,id_run+'_df_DistanceINDHenSim_DAILYLEVEL.csv'), sep=';',
                      parse_dates=['date1', 'date2'])
display(df_inter.head(3))
df_inter.shape

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date1,date2,HenID,chi2distance,nbr_obs_chi2all_30,li_chi2_30,chi2distance_ALL_30,DTW_30_1_Zone,nbr_obs_l1_30_1_Zone,nbr_obs_l2_30_1_Zone,...,DTW_30_3_Zone,nbr_obs_l1_30_3_Zone,nbr_obs_l2_30_3_Zone,DTW_30_4_Zone,nbr_obs_l1_30_4_Zone,nbr_obs_l2_30_4_Zone,DTW_30_5_Zone,nbr_obs_l1_30_5_Zone,nbr_obs_l2_30_5_Zone,DTW_30_all
0,2020-10-04,2020-10-05,hen_130,0.022121,19,"[0.9999999999989999, 0.024765157983628217, 0.0...",0.061993,0.0,19,19,...,14.7,19,19,1.1,19,19,21.9,19,19,40.0
1,2020-10-04,2020-10-06,hen_130,0.005760,19,"[0.2782426778237291, 0.06932523228813971, 0.0,...",0.026351,0.0,19,19,...,5.9,19,19,4.4,19,19,13.1,19,19,25.7
2,2020-10-04,2020-10-07,hen_130,0.009900,19,"[0.661454792656337, 0.024765157983628217, 0.0,...",0.044175,0.0,19,19,...,10.9,19,19,2.4,19,19,13.4,19,19,30.9


(2796681, 23)

In [9]:
li_dist = ['DTW_30_all', 'DTW_30_1_Zone', 'DTW_30_2_Zone', 'DTW_30_3_Zone', 'DTW_30_4_Zone', 'DTW_30_5_Zone']
df_inter.groupby(['HenID'])[['date1','date2']+li_dist].agg(lambda x: list(x)).reset_index()

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,HenID,date1,date2,DTW_30_all,DTW_30_1_Zone,DTW_30_2_Zone,DTW_30_3_Zone,DTW_30_4_Zone,DTW_30_5_Zone
0,hen_1,"[2020-09-30 00:00:00, 2020-09-30 00:00:00, 202...","[2020-10-01 00:00:00, 2020-10-02 00:00:00, 202...","[203.5, 240.1, 176.3, 228.0, 238.8, 238.8, 226...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[81.3, 95.7, 50.4, 95.7, 95.7, 95.7, 91.2, 95....","[21.7, 37.4, 37.5, 26.7, 36.3, 36.3, 31.9, 36....","[8.6, 8.6, 20.1, 9.4, 8.6, 8.6, 15.3, 8.6, 8.6...","[91.9, 98.4, 68.3, 96.2, 98.2, 98.2, 88.0, 98...."
1,hen_10,"[2020-09-30 00:00:00, 2020-09-30 00:00:00, 202...","[2020-10-01 00:00:00, 2020-10-02 00:00:00, 202...","[292.7000000000001, 298.4000000000001, 298.400...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[74.3, 74.3, 74.3, 74.3, 39.7, 11.4, 8.9, 31.2...","[84.0, 84.0, 84.0, 76.2, 87.3, 55.7, 74.1, 66....","[16.0, 18.9, 18.9, 7.8, 28.5, 58.6, 49.5, 49.1...","[118.4, 121.2, 121.2, 110.5, 97.9, 38.3, 71.4,..."
2,hen_100,"[2020-10-24 00:00:00, 2020-10-24 00:00:00, 202...","[2020-10-25 00:00:00, 2020-10-26 00:00:00, 202...","[226.1, 224.8, 234.1, 301.1, 218.3, 211.7, 210...","[20.2, 32.7, 33.0, 31.3, 18.5, 20.8, 22.7, 15....","[53.0, 45.2, 48.0, 90.6, 44.3, 39.9, 38.1, 43....","[40.5, 34.8, 37.2, 48.6, 46.8, 40.7, 53.3, 56....","[40.6, 47.0, 54.1, 57.1, 42.6, 41.7, 21.5, 37....","[71.8, 65.1, 61.8, 73.5, 66.1, 68.6, 75.1, 77...."
3,hen_101,"[2020-09-30 00:00:00, 2020-09-30 00:00:00, 202...","[2020-10-01 00:00:00, 2020-10-02 00:00:00, 202...","[88.0, 79.80000000000003, 111.1, 107.6, 149.5,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.4, 18.6,...","[43.5, 33.1, 46.0, 36.7, 48.2, 25.7, 48.0, 49....","[44.5, 45.7, 38.4, 38.8, 36.4, 27.8, 27.9, 34....","[0.0, 1.0, 1.5, 8.9, 21.9, 3.7, 45.0, 22.9, 32...","[0.0, 0.0, 25.2, 23.2, 43.0, 13.4, 18.1, 0.0, ..."
4,hen_102,"[2020-09-30 00:00:00, 2020-09-30 00:00:00, 202...","[2020-10-01 00:00:00, 2020-10-02 00:00:00, 202...","[107.0, 107.0, 107.0, 140.5, 107.0, 107.0, 107...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, ...","[48.0, 48.0, 48.0, 48.6, 48.0, 48.0, 48.0, 48....","[10.4, 10.4, 10.4, 32.3, 10.4, 10.4, 10.4, 10....","[43.5, 43.5, 43.5, 54.5, 43.5, 43.5, 43.5, 43...."
...,...,...,...,...,...,...,...,...,...
223,hen_95,"[2020-09-30 00:00:00, 2020-09-30 00:00:00, 202...","[2020-10-01 00:00:00, 2020-10-02 00:00:00, 202...","[159.4, 181.4, 182.0, 210.4, 222.5, 188.2, 219...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.3, 17.2, 7.9,...","[41.0, 22.3, 22.9, 35.0, 50.0, 55.2, 59.9, 52....","[46.4, 25.8, 42.5, 30.1, 42.3, 35.9, 40.8, 38....","[50.0, 56.4, 56.9, 66.9, 50.4, 16.0, 31.2, 39....","[22.0, 76.9, 59.7, 78.4, 79.8, 81.1, 79.3, 81...."
224,hen_96,"[2020-10-01 00:00:00, 2020-10-01 00:00:00, 202...","[2020-10-02 00:00:00, 2020-10-03 00:00:00, 202...","[257.1, 253.4, 244.0, 255.9, 256.6, 260.3, 303...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[121.2, 126.7, 119.4, 126.7, 126.6, 126.7, 126...","[10.8, 0.0, 0.0, 2.1, 2.4, 0.0, 4.0, 2.2, 0.0,...","[3.0, 0.0, 2.0, 1.7, 1.5, 0.0, 66.9, 0.0, 2.1,...","[122.1, 126.7, 122.6, 125.4, 126.1, 133.6, 106..."
225,hen_97,"[2020-09-30 00:00:00, 2020-09-30 00:00:00, 202...","[2020-10-01 00:00:00, 2020-10-02 00:00:00, 202...","[133.0, 153.8, 133.0, 101.4, 110.6, 136.3, 133...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[30.8, 33.4, 30.8, 30.8, 8.8, 30.8, 30.8, 30.8...","[18.8, 18.9, 18.8, 17.1, 37.5, 19.5, 18.8, 18....","[31.2, 40.2, 31.2, 25.9, 19.1, 32.2, 31.2, 31....","[52.2, 61.3, 52.2, 27.6, 45.2, 53.8, 52.2, 52...."
226,hen_98,"[2020-09-30 00:00:00, 2020-09-30 00:00:00, 202...","[2020-10-01 00:00:00, 2020-10-02 00:00:00, 202...","[150.9, 155.7, 173.1, 129.6, 194.2, 176.899999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[58.6, 59.0, 48.2, 30.6, 58.5, 51.8, 66.9, 65....","[40.1, 27.3, 44.3, 40.3, 48.6, 36.0, 53.1, 45....","[8.6, 9.7, 26.8, 13.3, 22.5, 23.8, 26.6, 35.9,...","[43.6, 59.7, 53.8, 45.4, 64.6, 65.3, 76.2, 87...."


In [7]:
#TOCHANGE!
dico_distance_mindist = {'DTW_30_all':175, 
                         'DTW_30_1_Zone':20, 
                         'DTW_30_2_Zone':40, 
                         'DTW_30_3_Zone':30,
                         'DTW_30_4_Zone':30,
                         'DTW_30_5_Zone':60}
li_df = []
li_date = set(df_daily['level'].tolist())

for d in tqdm.tqdm(sorted(list(li_date))):
    
    df_graph = df_inter[df_inter['date']==d].copy()
    
    if df_graph.shape[0]>0:
    
        #one graph per distance
        for D,min_dist in dico_distance_mindist.items():
            #initialise graph
            G = nx.Graph()
            #assign graph attributes (i.e. similarity computation)
            G.graph['similarity computation']= D

            #add nodes (s: with attributes if needed)
            li_hen_d = df_daily[df_daily['level']==d]['HenID'].unique()
            for h in li_hen_d:
                #s = max(df_movement[df_movement['HenID']==h]['severity'].tolist())
                G.add_node(h) #attributes:, max_severity=int(round(s,0)))

            #add edges
            li_edges = [] 
            #we use a symmetric measure, so we only do each combination once
            for i,h1 in enumerate(li_hen_d):
                #lets compute for hen h1 its difference with all other birds one by one
                for h2 in li_hen_d[i+1:]:
                    try:
                        dist = df_graph[df_graph['hen_pair']=='-'.join(sorted([h1,h2]))][D].values[0]
                        #link if dist < min_dist
                        if dist<min_dist:
                            li_edges.append((h1, h2, {'similarity':int(round(dist,2))}))     
                    except Exception as e:
                        print('------------------')
                        print(e)
                        print('-'.join(sorted([h1,h2])))
                        print(d)
            G.add_edges_from(li_edges)

            #save G (networkX graph) in GEXF format for gephi
            nx.write_gexf(G, os.path.join(path_extracted_data_SNA,D+str(d).split(' ')[0]+'.gexf'))

            #centrality measures
            #returns a Dictionary of nodes with degree centrality as the value
            #The degree centrality values are normalized by dividing by the maximum possible degree in a simple graph n-1 where n is 
            #the number of nodes in G.
            dico_v_dg = nx.degree_centrality(G)
            #Compute the shortest-path betweenness centrality for nodes. see doc for additional info
            #return a Dictionary of nodes with betweenness centrality as the value.
            dico_v_bw = nx.betweenness_centrality(G, seed=0)
            #Compute closeness centrality for nodesee doc for additional info
            #return a Dictionary of nodes with closeness centrality as the value
            dico_v_cl = nx.closeness_centrality(G)
            #into a datfarme
            df_ = pd.DataFrame(list(dico_v_cl.items()),columns = ['HenID','closeness'])
            df_['betweeness'] = df_['HenID'].map(lambda x: dico_v_bw[x])
            df_['degree'] = df_['HenID'].map(lambda x: dico_v_dg[x])
            df_['date'] = d  
            df_['distance_measure'] = D
            print(df_.shape)
            li_df.append(df_)
    else:
        print('NO DATAAAA')
        print(d)
df_SNA = pd.concat(li_df)
print(df_SNA.shape)
df_SNA.to_csv(os.path.join(path_extracted_data ,id_run+'_df_SNA.csv'), sep=';', index=False)
df_SNA.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'df_daily' is not defined